This notebook has the implementation of the Lattice Boltzmann algorithm for a fluid moving in the x-direction with periodic boundary conditions on this axis but no-slip boundary conditions in y -axis. The fluid is simulated as a  square lattice which each nodes can move in 9 different directions (d2q9 model) and the points that cross the top and bottom boundaries (y direction) are bounced back to the fluid restoring the previous position but with minus the velocity it had. The velocity in the x-direction will be constant and no object was put inside the fluid.

The program is divided as follows:

1- Import of the main libraries

2- Definition of the Globlal constants that characterize the fluid

3- Equilibrium function definition

4- Main time loop of the algorithm

Leila Inigo de la Cruz

In [2]:
import numpy as np
import math as math
import matplotlib.pyplot as plt
import statistics as stats
from numpy.linalg import *
from matplotlib import cm
import pylab


# Flow global constants  

In [40]:

maxIter = 250 # Total number of time iterations.
q = 9                            # amount of discrete velocities   
nx = 80; ny = 40;              # Lattice dimensions and population

# Fluid constants
Re      = 220.0                     # Reynolds number.        
r=ny/9;                            # characteristic length        
uLB     = 0.0005                    # Velocity in lattice units of the lattice.
vis    = uLB*r/Re                  # viscosity
# tau=  3.*vis+0.5               # Relaxation parameter
tau=0.7

# Velocities and weights d2q9

In [8]:
# Possible directions.
ei= np.array([(x,y) for x in [0,-1,1] for y in [0,-1,1]])

# Lattice weights.
# w = 1/9 * np.ones(q) 
# for i in range(q):
#     if i%2==0:
#         w[i]=1/36
# w[0] = 4/9 

weights = 1/9*np.ones(q)
weights[0] = 4/9
weights[2::2] = 1/36        

# Equilibrium Function Definition

In [106]:
# This function calculates the equilibrium distribution of the density of particles in the lattice 
#with certain speed , at certain position and time.
#inputs:
# u- speed of each particles, defined as an array of three dimensions:nx,ny,2,namely, the size of the lattice and 
#    the two components in x and y.
# ei- the directions that can take the points in the lattice, defined as an array of 9 vectors of two dimensions.
#rho- the density of the fluid, defined as a number result of sum of the density over certain direction of the velocity
#outputs:
# feq- The equilibrium distribution as an array of three dimensions: 9, nx,ny

def equilibrium(u,ei,rho): 

    #     # term by term of the equilibrium density
    term1=3*np.inner(ei, u.transpose(1,0,2)).transpose(0,2,1)
    #     term1 = 3.0 * np.dot(ei,u.transpose(0,2,1))
    term2=0.5*term1**2
    #     term2 = 0.5 *term1**2
    term3=3/2*(u[:,:,0]**2+u[:,:,1]**2)
    
#     # Fill in the density term by term in each possible direction given by the q variable.
    
    for i in range(q):
        feq[i,:,:]= rho*w[i]*(1.+term1[i,:,:]+term2[i,:,:]-term3)

    return feq



# Main time loop

In [117]:

#initialization of the equilibrium density with an extra layer of zeros at the top and the bottom

feq=np.ones(shape=(q,nx,ny+1))
flowin=feq.copy()

for time in range(maxIter):
    

    for i in range(q):
        
    #     Rolling of the grid 
        flowin[i, :, :] = np.roll(flowin[i, :, :], ei[i,0], axis = 0)
        flowin[i, :, :] = np.roll(flowin[i, :, :], ei[i,1], axis = 1)

#         flowin[i,:,:] = np.roll(np.roll(feq[i,:,:],ei[i,0],axis=0),ei[i,1],axis=1)
#   Bounce back boundary conditions

    flowin[1:q,:,0]=np.roll(flowin[1:q,:,0],4,axis=0)
    flowin[1:q,:,ny]=np.roll(flowin[1:q,:,ny],4,axis=0)
    
    rho = np.sum(flowin,axis=0)     # Calculate macroscopic density 
    
    u = [np.inner(flowin.transpose(2,1,0),ei[:, 0]).transpose(), np.inner(flowin.transpose(2,1,0),ei[:, 1]).transpose() ]/ rho
#     u = np.dot(ei.transpose(), flowin.transpose((1,0,2)))/rho
    u=u.transpose(1,2,0)
    u[:,:,0] +=uLB # Adding constant velocity in x direction
    
    
   
    
#         if np.any(flowin[i,:,ny]!=np.zeros([nx-1,1])):
#             flowin[i,:,ny]=flowin[i-1,:,ny]
# #             move the particles that moved to that layer back and reflect them on the y direction
#             u[:,ny,1]=-u[:,ny,1]*ei[i,1]
    

#         if np.any(flowin[i,:,0]!=np.zeros([nx-1,1])):
#             flowin[i,:,0]=flowin[i-1,:,0]
# #             move the particles that moved to that layer back and reflect them on the y direction
#             u[:,0,1]=-u[:,0,1]*ei[i,1]
        
        
       
    feq = equilibrium(u,ei,rho)  
    
    flowin = flowin - (1/tau)*(flowin-feq)
#     flowin = ((tau-1)/tau)*flowin -(flowin - feq)/tau  # evolution equation 

        
   
  

In [118]:
x=np.linspace(0,ny,ny-1)

plt.scatter(x,u[2,0:ny-1,0])
plt.plot(x,u[2,0:ny-1,0])
plt.xlabel('Lattice points in y')
plt.ylabel('Velocity in x direction')
pylab.show()